#### Handling missing attributes at training
If only a small fraction of data points have missing attributes and the amount of data at hand is very large one might as well exclude such *deficient* data points during training. This however is wasteful and is often times a luxury one cannot afford.  
A common way of handling missing data during training is by simply computing impurities (GNI, entropy, etc.) at nodes using whatever information is available, i.e. by ignoring missing information. To illustrate, consider a toy dataset below with 5 data points and 3 features. Data point $x_1$ is missing feature $f_2$, $x_2$ is missing $f_1$ and $x_4$ is missing both $f_2$ and $f_3$.

|        | $f_1$| $f_2$| $f_3$| 
|--------|------|------|------|
|  $x_1$ | 12.23|      | 0.32 |
|  $x_2$ |      |78.22 | 0.77 |
|  $x_3$ | 10.45|61.22 | 0.52 |
|  $x_4$ | 8.89 |      |      |
|  $x_5$ | 9.66 | 32.2 | 1.62 |


Now while computing split for feature $f_1$ all but data point $x_2$ will be used since $x_2$ is missing feature $f_2$. Similarly for $f_2$, $x_1$ and $x_4$ will be excluded. Of course the impurities will be computed with different number of samples for different features but this is better than discarding *deficient* data points altogether. Do you see why?


#### Handling missing attributes at inference
In this section we will see how use of surrogate splits will allow us to handle missing attributes during inference. Surrogate split acts as a proxy for the primary split in case the data point is missing attribute which the primary split uses for making decision. Suppose the primary split at some node $N$ uses feature $f_i$ to decide which way a data point goes down the tree (left or right subtree) and suppose we encounter a data point for which that feature $f_i$ is missing. What would we do in such a case? The way *CART*s handle such situation is by maintaining a series of surrogate or substitute splits. Such surrogate splits are ordered by how well they approximate the primary split, often know as **predictive association**. Splits are tried in order (first the primary and then surrogate with best predictive association with the primary and then surrogate with second best association and so on) and the first split for which the data point has value for is used. 

**Predictive association** is simply the number of samples sent to the right and to the left by both splits. The goal of surrogate split here is to approximate primary split as close as possible. In doing so the surrogate split might however choose a threshold which might not be the optimal threshold in terms of reduction in impurity for that feature. 

Also it isn't true that a new subtree is grown for each surrogate split. That would be computationally prohibitive since that would require growing exponential subtrees. Why? 

Nor it is that the surrogate splits are second best split, third best split and so on in term of impurity reduction. Surrogate splits simply route data roughly the same way as primary split but using different attributes and thresholds. Which also means surrogate splits might route few data points wrongly (to the right subtree instead of left and vice versa) compared to the primary split. To drive the concept home lets work through the following toy example$^{1}$: 


|            | $f_1$ | $f_2$ | $f_3$ | y        |
|------------|------ |-------|-------|----------|
|  $x_1$    |   0   |   7   |   8   |  $c_1$   |
|  $x_2$     |   1   |   8   |   9   |  $c_1$   |
|  $x_3$    |   2   |   9   |   0   |  $c_1$   |
|  $x_4$    |   4   |   1   |   1   |  $c_1$   |
| $x_5$    |   5   |   5   |   2   |  $c_1$   |
|  $x_6$     |   3   |   3   |   3   |  $c_2$   |
|  $x_7$     |   6   |   0   |   4   |  $c_2$   |
|  $x_8$     |   7   |   4   |   5   |  $c_2$   |
|  $x_9$     |   8   |   5   |   6   |  $c_2$   |
|  $x_{10}$  |   9   |   6   |   7   |  $c_2$   |

Lets first order samples by attribute values:



|            |   0   |   1   |   2   |   3   |   4   |   5   |   6   |  7    |   8   |   9   |   
|------------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|
|  $f_1$     | <font color='blue'>$x_1$</font> | <font color='blue'>$x_2$</font> | <font color='blue'>$x_3$</font> | <font color='green'>$x_6$</font> | <font color='blue'>$x_4$</font> | <font color='blue'>$x_5$ </font>| <font color='green'>$x_7$</font> | <font color='green'>$x_8$</font> | <font color='green'>$x_9$</font> | <font color='green'>$x_{10}$</font>|
|  $f_2$     | <font color='green'>$x_7$</font> | <font color='blue'>$x_4$</font> | <font color='blue'>$x_5$</font> | <font color='blue'>$x_6$</font> | <font color='green'>$x_8$</font> | <font color='green'>$x_9$</font> |<font color='green'>$x_{10}$</font>| <font color='blue'>$x_1$</font> | <font color='blue'>$x_2$</font> | <font color='blue'>$x_3$</font> |
|  $f_3$     | <font color='blue'>$x_3$</font> | <font color='blue'>$x_4$</font> | <font color='blue'>$x_5$ </font>| <font color='green'>$x_6$</font> | <font color='green'>$x_7$</font> | <font color='green'>$x_8$</font> | <font color='green'>$x_9$</font> | <font color='green'>$x_{10}$</font>| <font color='blue'><font color='blue'>$x_1$</font> | <font color='blue'>$x_2$</font> |


It is obvious from above that $f_1 < 5.5$ provides best split of data in terms of impurity. Hence $f_1$ will be our primary split. 

Now among $f_2$ and $f_3$ which one forms a better surrogate split to $f_1 < 5.5$? 

As can be verified,  $f_3 < 3.5$ is the best surrogate split. Why? (*Hint* : Go through each possible split for each attribute and find the split that best approximates the primary split)

 
<img src='surrogate_split.svg'  width="800">

How about error? How does generalization error evolve as a function of number of missing values in the data? (*Hint*: More the missing values more the use of surrogate splits during inference) 

Finally, can you guess which attribute is going to be picked as the best surrogate split based on correlation information between attributes? 

##### References :
[1] Examples and figure derived from *Chapter : 8.3.10 Missing attributes*, Pattern Classification, 2nd Edition : Richard O. Duda, Peter E. Hart, David G. Stork